In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
vd = cv2.VideoCapture("opencvHW.mp4")#檔案路徑

fps=vd.get(cv2.CAP_PROP_FPS)
effect=fps*4
effect2=fps*8
effect3=fps*12
effect4=fps*16
effect5=fps*20
effect6=fps*24
effect7=fps*28
effect8=fps*32
effect9=fps*36
effect10=fps*40
effect11=fps*44

start=0
ret, frame = vd.read()
if not ret:
    print("無法讀取")
    vd.release()
    exit()
height, width ,_= frame.shape
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width,height))
vd.set(cv2.CAP_PROP_POS_FRAMES, 0)
while True:
    # ret 是一個布爾值，表示是否成功讀取了幀
    # frame 是讀取到的每一幀圖像
    effect_text=''
    ret, frame = vd.read()
    # 如果成功讀取到幀，ret 將為 True
    if not ret:
        print("無法讀取視頻或視頻播放完畢。")
        break
    # 翻轉180
    if  start>=effect and start < effect2:   
        frame = cv2.rotate(frame, cv2.ROTATE_180)
        effect_text='rotate_180'
    # 高斯濾波
    elif start>=effect2 and start < effect3:
        frame =cv2.GaussianBlur(frame, (3, 3), .5, .5)
        effect_text='GaussianBlur'
    #bgr重新排序 
    elif start>=effect3 and start < effect4:
        b,g,r=cv2.split(frame)
        frame =cv2.merge([r,g,b])
        effect_text='bgr merge rgb'
    # 邊緣偵測
    elif start>=effect4 and start < effect5:
        sobelx = cv2.Sobel(frame, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(frame, cv2.CV_64F, 0, 1)
        
        sobelx = cv2.convertScaleAbs(sobelx)   # 轉回 uint8 
        sobely = cv2.convertScaleAbs(sobely)  
        frame =  cv2.addWeighted(sobelx, .5, sobely, .5, 0)
        effect_text='Edge detection'
    # 灰階圖像二值化
    elif start>=effect5 and start < effect6:
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, frame = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)  # 1 : 255, 0 : 0
        frame=cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        effect_text='Threshold'
    # 侵蝕
    elif start>=effect6 and start < effect7:
        kernel = np.ones((3, 3),np.uint8)  # try  [3, 3]
        frame = cv2.erode(frame, kernel, iterations = 3)
        effect_text='Erode'
    # 膨脹
    elif start>=effect7 and start < effect8:
        kernel = np.ones((3, 3),np.uint8)  # try  [3, 3]
        frame = cv2.dilate(frame, kernel, iterations = 3)
        effect_text='Dilate'
    #開運算 先侵蝕再膨脹
    elif start>=effect8 and start < effect9:
        frame=cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
        sift = cv2.xfeatures2d.SIFT_create()
        effect_text='Eorphology open'
    # sift劃出特徵
    elif start>=effect9 and start < effect10:
        kp, des = sift.detectAndCompute(frame, None)   
        frame = cv2.drawKeypoints(frame, kp, None)
        effect_text='Sift'
    #形態學梯度 膨脹-侵蝕 得到輪廓 
    elif start>=effect10 and start < effect11:
        frame = cv2.morphologyEx(frame, cv2.MORPH_GRADIENT, kernel)
        effect_text='Morphology grandient'
        
    #灰階彩色 全域性直方圖均衡 
    elif start>=effect11 :
        img=frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        height, width = gray.shape
        (b, g, r) = cv2.split(img)        
        b = cv2.equalizeHist(b)
        g= cv2.equalizeHist(g)
        r = cv2.equalizeHist(r)
        frame3 = cv2.merge((b, g, r))
        img = cv2.resize(img, (width // 2, height//2))
        frame3 = cv2.resize(frame3, (width // 2, height//2))
        colormix = np.hstack((img, frame3))
        frame2 = cv2.equalizeHist(gray) 
        gray = cv2.resize(gray, (width // 2, height//2))
        frame2 = cv2.resize(frame2, (width // 2, height//2))
        frame2 = cv2.equalizeHist(gray)
        graymix = np.hstack((gray, frame2))
        graymix=cv2.cvtColor(graymix, cv2.COLOR_GRAY2BGR)

        frame = np.vstack((graymix,colormix))
        effect_text='Gray and Color Histogram Equalization'
        
        

        
        
        
        
    
    # else:
    # # 顯示幀
    cv2.putText(frame, effect_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    out.write(frame)
    cv2.imshow('Video Frame', frame)
    
    # 等待 25 ms，並檢查是否有按鍵按下，如果按下 'q'，則退出循環
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    start+=1
    

# 釋放 VideoCapture 對象並關閉窗口
vd.release()
cv2.destroyAllWindows()
cv2.waitKey(1) 


無法讀取視頻或視頻播放完畢。


-1